# Proyecto 1: Clock App

Los temas más importantes que trataremos en este proyecto son:
* La sintáxis básica del lenguaje Kivy.
* Llamar y ubicar componentes de Kivy.
* Hacer que una aplicación esté pendiente de los eventos.
* Cargar diferentes fuentes y dar formato al texto.

Ahora que sabemos como crear la estructura base de una app crearemos nuestra primera aplicación seria. Se trata de un reloj con cronómetro. Para comenzar crearemos la estructura básica como ya sabemos hacerlo. El programa principal en Python (llamado `main.py`) y el archivo auxiliar en lenguaje KV llamado clock.kv, ya que nuestra aplicación se llamará ClockApp. Recordemos que el fichero kv debe llevar de nombre el mismo que la app pero quitándole la App final y comenzando con minúscula.

El archivo principal será:

In [ ]:
# File: main.py
from kivy.app import App

class ClockApp(App):
    pass

if __name__ == '__main__':
    ClockApp().run()

Y el archivo `clock.kv`:

In [ ]:
# File: clock.kv
BoxLayout:
    orientation: 'vertical'
        
    Label:
        text: '00:00:00'

Hasta ahora la aplicación se comporta como nuestra app anterior "Hello World!". El contenedor `BoxLayout` permite que dos o más widgets que tenga en su interior puedan coexistir uno al lado del otro. Es como un recipiente y los widgets que tienga en su interior diremos que son hijos suyos. Si solo contiene un único widget, como en nuestro caso, `BoxLayout` llenará todo el espacio disponible en la pantalla con él. Como `Label` es el widget raíz ha llenado toda la ventana de la aplicación con la etiqueta.

## Haciendo que el reloj funcione

Kivy es un lenguaje basado en eventos, es decir deberá poder ser capaz de estar atento a los posibles eventos producidos por el usuario. Por este motivo Kivy tendrá un bucle principal y después de realizar alguna tarea deberá volver a él para seguir "escuchando" posibles eventos. Por ese motivo no debemos hacer nunca que una terea específica de la aplicación entre en un bucle infinito porque eso hará que nuestra aplicación no responda a los eventos del usuario y se "cuelgue". Por eso necesitamos que una arquitectura basada en eventos esté siempre lista para responder a eventos de distintos tipos, como entradas del usuario, eventos de red o finalizaciones.

Uno de los eventos más comunes a los que los programas suelen escuchar es `App.on_start`. Un método con este nombre se define en la clase de la aplicación y será llamado tan pronto como la aplicación sea llamada. Otro evento que usaremos comunmente será `on_press`, que se dispara cuando el usuario presiona sobre un botón.

Para acceder al widget `Label` que escribe el valor del tiempo le daremos un identificador único (`id`). Posteriormente nos será más fácil acceder a los widgets a través de estos identificadores.

Modifiquemos nuestro archivo `clock.kv` agregando:

In [ ]:
Label:
    id: time

Ahora podremos accder a `Label` desde nuestro código directamente usando la notación `root.ids.time` (`root` en nuestro caso es `BoxLayout`).

Para actualizar la hora agregaremos a nuestra clase `ClockApp` el método `update_time` que se verá así:

In [ ]:
def update_time(self,nap):
    self.root.ids.time.text = strftime('%H:%M:%S')

La función `strftime` viene en el módulo `time` de la biblioteca estándar, ppor lo tanto deberemos importarla agregando a las primeras líneas de nuestro código:

In [ ]:
from time import strftime

Haremos ahora que la función de actualización se ejecute una vez por segundo una vez iniciado el programa:

In [ ]:
def on_start(self):
    Clock.schedule_interval(self.update_time, 1)

El método `Clock.schedule_interval` ejecuta una función periodicamente, en este caso `update_time`. Para poder usar este método deberemos importar la clase `Clock` de Kivy poniendo entre las líneas de importación:

In [ ]:
from kivy.clock import Clock

Si ejecutamos nuestra aplicación ahora veremos como... ¡el reloj funciona!

## Disposición de los objetos: Layout

Para disponer los widgets en la pantalla, Kivy provee una serie de clases `Layout`. `Layout` es una subclase de `Widget` y sirve como un contenedor para otros widgets.Para nuestra aplicación queremos una disposición sencilla, como la de la figura de abajo.

<img src="layout.png">

Para nuestra aplicación ya hemos utilizado el `BoxLayout`, que es en esencia una grilla unidimensional. Como por ahora solo hemos puesto dentro un único elemento éste no se ve afectado por otra cosa.

Los layouts de Kivy intentan llenar toda la pantalla, por lo que nuestra aplicación se adaptará al cambio de medida y orientación de la pantalla de manera automática.

Si añadimos otro BoxLayout a nuestra pantalla, tomará la mitad del tamaño de la misma, dependiendo de la orientación: un boxlayout vertical crecerá de abajo hacia arriba, y una horizontal de izquierda a derecha.

Mirando la dispocición que queremos adoptar seguramente habrás adivinado que si queremos alinear los botones horizontalmente deberemos incluir dentro del primer boxlayout otro horizontal. Los layouts y los widgets pueden anidarse unos dentro de otros de manera reativa para lograr interfaces complejas.

## Finalizando el Layout

Ahora agregaremos los widgets a nuestro reloj. Ya teníamos un `BoxLayout` vertical con una etiqueta (`Label`) dentro. Ahora agregaremos un `Boxlayout` horizontal que contendrá nuestros dos botones en su interior y una segunda etiqueta que mostrará nuestro cronómetro. El código que aparece abajo es autoexplicativo.

Si probamos este código veremos que funciona, aunque no se ve muy bonito, luego veremos como mejorar su aspecto. 